# Previsão de Estoque para 28 dias

In [8]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Google Cloud Platform
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

from datetime import timedelta

# Caminho para o arquivo de chave da conta de serviço
service_account_path = 'tfm-sa.json'

# Criar objeto de credenciais
credentials = service_account.Credentials.from_service_account_file(service_account_path)

# Configurações do projeto e dataset
project_id = 'perseverance-332400'
dataset_id = 'TFM'
table_id = 'ds_market'
full_table_id = f'{project_id}.{dataset_id}.{table_id}'

# Função para configurar o cliente BigQuery usando credenciais específicas
def initialize_bigquery_client():
    return bigquery.Client(project=project_id, credentials=credentials)


# Função para configurar o cliente BigQuery
def initialize_bigquery_client():
    return bigquery.Client()

# Função para extrair dados históricos do BigQuery
def get_historical_data_from_bigquery(query: str, client) -> pd.DataFrame:
    query_job = client.query(query)
    data = query_job.to_dataframe()
    print(f"Dados extraídos do BigQuery com {data.shape[0]} linhas e {data.shape[1]} colunas.")
    return data

# Função para preparar os dados
def prepare_stock_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    # Converter coluna de data
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    # Ordenar por data e remover duplicatas
    cleaned_data = raw_data.drop_duplicates().sort_values('date').set_index('date')
    # Preencher valores ausentes (exemplo: preenchimento com média)
    cleaned_data['stock'] = cleaned_data['stock'].fillna(cleaned_data['stock'].mean())
    print("Dados preparados e limpos.")
    return cleaned_data

# Função para treinar o modelo ARIMA
def train_forecast_model(data: pd.DataFrame, order=(1, 1, 1)):
    print("Treinando o modelo ARIMA...")
    model = ARIMA(data['stock'], order=order)
    model_fit = model.fit()
    print("Modelo treinado com sucesso.")
    return model_fit

# Função para gerar previsões
def predict_stock(data: pd.DataFrame, model, steps=30) -> pd.DataFrame:
    print(f"Gerando previsões para {steps} dias...")
    forecast = model.get_forecast(steps=steps)
    forecast_df = forecast.summary_frame()
    # Criar uma coluna de datas para as previsões
    forecast_df['date'] = pd.date_range(data.index[-1] + timedelta(days=1), periods=steps)
    forecast_df = forecast_df.reset_index(drop=True)
    forecast_df = forecast_df[['date', 'mean']]
    forecast_df.columns = ['date', 'forecast_stock']
    print("Previsão gerada com sucesso.")
    return forecast_df

# Função para carregar previsões no BigQuery
def store_forecast_results(data: pd.DataFrame, table_name: str, client):
    print(f"Carregando previsões para a tabela {table_name} no BigQuery...")
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(data, table_name, job_config=job_config)
    job.result()  # Espera o job terminar
    print("Previsões carregadas com sucesso no BigQuery.")

# Pipeline completo para previsão de estoque
def run_forecast_pipeline(query: str, table_name: str, steps=30, order=(1, 1, 1)):
    client = initialize_bigquery_client()
    
    # 1. Extrair dados históricos do BigQuery
    raw_data = get_historical_data_from_bigquery(query, client)
    
    # 2. Preparar dados para modelagem
    prepared_data = prepare_stock_data(raw_data)
    
    # 3. Treinar o modelo de previsão e fazer previsões
    model = train_forecast_model(prepared_data, order=order)
    forecast = predict_stock(prepared_data, model, steps=steps)
    
    # 4. Carregar previsões no BigQuery
    store_forecast_results(forecast, table_name, client)

# Exemplo de execução do pipeline
query = """
    SELECT date, stock
    FROM `your_dataset.stock_data`
    ORDER BY date ASC
"""
table_name = "your_dataset.stock_forecast"

run_forecast_pipeline(query, table_name, steps=30, order=(1, 1, 1))


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.